In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import random

import pandas as pd

from torch.utils.data import Dataset
import torch.nn.functional as F
import torch.nn as nn

from torch.optim import AdamW

import torch_geometric.transforms as T

from torch_geometric.data import Batch

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import global_add_pool
from torch_geometric.nn import GraphConv
from torch.utils.data import DataLoader

from pathlib import Path

from tqdm import tqdm

In [3]:
import sys
import os
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
parent_parent_dir = os.path.dirname(parent_dir)

sys.path.append(parent_dir)
sys.path.append(parent_parent_dir)

from DataPipeline.dataset import ZincSubgraphDatasetStep, custom_collate_GNN2
from Model.GNN2 import ModelWithEdgeFeatures
from Model.metrics import pseudo_accuracy_metric, pseudo_recall_for_each_class, pseudo_precision_for_each_class

In [4]:
datapath = Path('..') / '../DataPipeline/data/preprocessed_graph_no_I_Br_P.pt'
dataset = ZincSubgraphDatasetStep(data_path = datapath, GNN_type=2)

Dataset encoded with size 7


In [5]:
loader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=custom_collate_GNN2)

In [6]:
encoding_size = 7

model = ModelWithEdgeFeatures(in_channels=encoding_size, hidden_channels_list=[64, 128, 256, 512, 512], mlp_hidden_channels=512, edge_channels=4, num_classes=4, use_dropout=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set up the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=0.0001)

# Set up the loss function for multiclass 
criterion = nn.CrossEntropyLoss()


name = 'GNN2_test'

# Training function

from tqdm.notebook import tqdm as tqdm_notebook
import numpy as np

In [7]:

from sklearn.metrics import accuracy_score, recall_score, precision_score
def train(loader, epoch):
    model.train()
    total_loss = 0
    
    progress_bar = tqdm_notebook(loader, desc="Training", unit="batch")

    avg_output_vector = np.zeros(4)  # Initialize the average output vector
    avg_label_vector = np.zeros(4)  # Initialize the average label vector
    total_graphs_processed = 0

    num_correct = 0
    num_correct_recall = torch.zeros(4)
    num_correct_precision = torch.zeros(4)
    count_per_class_recall = torch.zeros(4)
    count_per_class_precision = torch.zeros(4)

    true_labels = torch.tensor([])
    predicted_labels = torch.tensor([])

    for batch_idx, batch in enumerate(progress_bar):
        data = batch[0]
        edge_infos = batch[1]
        data = data.to(device)
        optimizer.zero_grad()
        logit_out = model(data)
        edge_infos = edge_infos.to(device)

        loss = criterion(logit_out, edge_infos)
       
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
        loss_value = total_loss / (data.num_graphs * (progress_bar.last_print_n + 1))
        
        out = F.softmax(logit_out, dim=1)

        # Collect true labels and predicted labels
        num_correct += pseudo_accuracy_metric(out.detach().cpu(), edge_infos.detach().cpu(), random=True)

        recall_output = pseudo_recall_for_each_class(out.detach().cpu(), edge_infos.detach().cpu(), random=True)
        precision_output = pseudo_precision_for_each_class(out.detach().cpu(), edge_infos.detach().cpu(), random=True)
        num_correct_recall += recall_output[0]
        num_correct_precision += precision_output[0]
        count_per_class_recall += recall_output[1]
        count_per_class_precision += precision_output[1]

        current_avg_output_vector = avg_output_vector / total_graphs_processed
        current_avg_label_vector = avg_label_vector / total_graphs_processed
        
        # Update the average output vector
        avg_output_vector += out.detach().cpu().numpy().mean(axis=0) * data.num_graphs
        avg_label_vector += edge_infos.detach().cpu().numpy().mean(axis=0) * data.num_graphs
        total_graphs_processed += data.num_graphs
        current_avg_output_vector = avg_output_vector / total_graphs_processed
        current_avg_label_vector = avg_label_vector / total_graphs_processed
        avg_correct = num_correct / total_graphs_processed
        avg_correct_recall = num_correct_recall / count_per_class_recall
        avg_correct_precision = num_correct_precision / count_per_class_precision
        avg_f1 = 2 * (avg_correct_recall * avg_correct_precision) / (avg_correct_recall + avg_correct_precision)
        progress_bar.set_postfix(loss=loss_value, avg_output_vector=current_avg_output_vector, 
                                 avg_label_vector=current_avg_label_vector, 
                                 avg_correct=avg_correct, num_correct=num_correct, 
                                 total_graphs_processed=total_graphs_processed, 
                                 avg_correct_precision=avg_correct_precision, 
                                 avg_correct_recall=avg_correct_recall, 
                                 avg_f1=avg_f1,
                                 count_per_class_precision=count_per_class_precision,
                                 count_per_class_recall=count_per_class_recall)

    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)

    # Compute accuracy, recall, and precision for each class
    accuracy = accuracy_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels, average=None)
    precision = precision_score(true_labels, predicted_labels, average=None)

    return (
        total_loss / len(loader.dataset),
        current_avg_label_vector,
        current_avg_output_vector,
        accuracy,
        recall,
        precision
    )

In [8]:
training_history = pd.DataFrame(columns=['epoch', 'loss', 'avg_output_vector', 'avg_label_vector', 'avg_correct', 'precision', 'recall'])

n_epochs = 1000
for epoch in range(1, n_epochs+1):
    loss, avg_label_vector, avg_output_vector, avg_correct = train(loader, epoch)
    # Convert tensors to numpy arrays or single values for storage in the dataframe
    loss = loss.item() if torch.is_tensor(loss) else loss
    avg_output_vector = avg_output_vector.numpy() if torch.is_tensor(avg_output_vector) else avg_output_vector
    avg_label_vector = avg_label_vector.numpy() if torch.is_tensor(avg_label_vector) else avg_label_vector
    avg_correct = avg_correct.item() if torch.is_tensor(avg_correct) else avg_correct

    # Assuming precision and recall are calculated somewhere in your train function
    # Convert them to numpy or single values as well
    precision = precision.numpy() if torch.is_tensor(precision) else precision
    recall = recall.numpy() if torch.is_tensor(recall) else recall

    training_history.loc[epoch] = [epoch, loss, avg_output_vector, avg_label_vector, avg_correct, precision, recall]

    save_every_n_epochs = 5
    if (epoch) % save_every_n_epochs == 0:
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            # Add any other relevant information you want to save here
        }
        torch.save(checkpoint, f'checkpoint_epoch_{epoch+1}_{name}.pt')
        
    # Save the training history every 1 epochs
    if epoch % 1 == 0:
        training_history.to_csv(f"training_history_{name}.csv", index=False)
    print(f'Epoch: {epoch}, Loss: {loss:.8f}')

Training:   0%|          | 0/1845 [00:00<?, ?batch/s]

C:\Users\goupi\AppData\Local\Temp\ipykernel_14868\410524407.py:48: RuntimeWarning: invalid value encountered in divide
  current_avg_output_vector = avg_output_vector / total_graphs_processed
C:\Users\goupi\AppData\Local\Temp\ipykernel_14868\410524407.py:49: RuntimeWarning: invalid value encountered in divide
  current_avg_label_vector = avg_label_vector / total_graphs_processed


Epoch: 1, Loss: 0.43639229


c:\Users\goupi\.conda\envs\torch_geometric\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\goupi\.conda\envs\torch_geometric\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\goupi\AppData\Local\Temp\ipykernel_14868\3785573072.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training_history = training_history.append({'epoch': epoch, 'loss': loss, 'avg_output_vector': avg_output_vector, 'avg_label_vector': avg_label_vector, 'avg_correct': accuracy, 'precision': precision, 'recall': recall}, ignore_index=True)


Training:   0%|          | 0/1845 [00:00<?, ?batch/s]

...

C:\Users\goupi\AppData\Local\Temp\ipykernel_14868\410524407.py:48: RuntimeWarning: invalid value encountered in divide
  current_avg_output_vector = avg_output_vector / total_graphs_processed
C:\Users\goupi\AppData\Local\Temp\ipykernel_14868\410524407.py:49: RuntimeWarning: invalid value encountered in divide
  current_avg_label_vector = avg_label_vector / total_graphs_processed


Epoch: 110, Loss: 0.24925687


c:\Users\goupi\.conda\envs\torch_geometric\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\goupi\.conda\envs\torch_geometric\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\goupi\AppData\Local\Temp\ipykernel_14868\3785573072.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training_history = training_history.append({'epoch': epoch, 'loss': loss, 'avg_output_vector': avg_output_vector, 'avg_label_vector': avg_label_vector, 'avg_correct': accuracy, 'precision': precision, 'recall': recall}, ignore_index=True)


Training:   0%|          | 0/1845 [00:00<?, ?batch/s]

C:\Users\goupi\AppData\Local\Temp\ipykernel_14868\410524407.py:48: RuntimeWarning: invalid value encountered in divide
  current_avg_output_vector = avg_output_vector / total_graphs_processed
C:\Users\goupi\AppData\Local\Temp\ipykernel_14868\410524407.py:49: RuntimeWarning: invalid value encountered in divide
  current_avg_label_vector = avg_label_vector / total_graphs_processed
